In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2
from config import db_password,g_key
from config import resolution,lat_max,lat_min,lng_max,lng_min
from config import t_sat,flow_rate,project_span
import json
from shapely.geometry import Point,shape
import os
from datetime import datetime,timedelta
from functions import Rmax_calc,recovery_calc,month_index
import scipy.optimize
from geojson import Feature, FeatureCollection, Polygon

In [2]:
# blocks information
file_path = os.path.join('..','..','Resources','blocks_info.csv')
blocks_df = pd.read_csv(file_path)
# actual gold produciton by month
file_path = os.path.join('..','..','Resources','act_monthly_oz.csv')
act_oz_df = pd.read_csv(file_path)

In [3]:
# Convert to datetime format
blocks_df['stack_finish'] = pd.to_datetime(blocks_df['stack_finish'])
blocks_df['leach_start'] = pd.to_datetime(blocks_df['leach_start'])
blocks_df['leach_end'] = pd.to_datetime(blocks_df['leach_end'])
blocks_df['leach_days'] = blocks_df['leach_end']-blocks_df['leach_start']
lat_lng_list = blocks_df[['lat','lng']].values.tolist()
# Create a dataframe with sequence as block_id, lift and lat_lng as columns
block_leach_time = pd.DataFrame(blocks_df[['lift']])
block_leach_time['lat_lng']=blocks_df[['lat','lng']].values.tolist()
block_leach_time.index.name = 'block_id'
# Create block_list for later index reference.
block_list = block_leach_time.index.tolist()

# Set up project span
projectspan = (
    datetime.strptime(project_span['startdate'],'%Y-%m-%d'),
    datetime.strptime(project_span['enddate'],'%Y-%m-%d')
    )
# Create month list of the project
startyear = projectspan[0].year
startmonth = projectspan[0].month
endyear = projectspan[1].year
endmonth = projectspan[1].month
monthlist = [datetime(m//12, m%12+1, 1) for m in range(startyear*12+startmonth-1, endyear*12+endmonth)]
# Use monthlist create recovery dataframe
leach_days_df = pd.DataFrame(monthlist,columns = ['months'])
leach_ks_df = pd.DataFrame(monthlist,columns = ['months'])
leach_rec_df = pd.DataFrame(monthlist,columns = ['months'])

# Break dataframe to records
blocks_records = blocks_df.to_records(index=False)

In [4]:
# This block of code calculates a specific block's recovery over months.
## The function takes two block indices 
## and the k factor that is going to be used for recovery calculation.
def block_leach_days(i,ref,lift_diff):
    # if the two indices are the same, means this is calculating the primary leach recovery
    calc_block = blocks_records[i]
    if i == ref:
        start_date = pd.to_datetime(calc_block['leach_start'])
        end_date = pd.to_datetime(calc_block['leach_end'])
    # if the two indices are different, means calculating recoveries of blocks under secondary leach
    else:
        # ref_block is the block on the very top
        ref_block = blocks_records[ref]
        # the blocks below the ref_block inherit the leaching dates from the ref_block
        # calculate leach delay
        start_date = pd.to_datetime(ref_block['leach_start']) + timedelta(days = t_sat + round(lift_diff * flow_rate))
        end_date = pd.to_datetime(ref_block['leach_end']) + timedelta(days = round(lift_diff * flow_rate))
    # locate the month row
    m_index = month_index(start_date,projectspan)
    start_month = start_date.month
    end_month = end_date.month + (end_date.year - start_date.year) * 12
    # calcuate how much recovery that this block has achieved since the start
    for m in range(start_month,end_month+1):
        # calculate how many days in this month for recovery calculation
        if m == start_month:
            if m == 12:
                days = 31 - start_date.day
            else:
                days = (datetime(start_date.year,start_date.month+1,1)-start_date).days
        elif m == end_month:
            days = end_date.day
        elif m < 12:
            days = (datetime(start_date.year,m+1,1)-datetime(start_date.year,m,1)).days
        elif m == 12:
            days = 31
        else:
            days = (datetime(start_date.year+1,m-12+1,1)-datetime(start_date.year+1,m-12,1)).days
        # write the recovery to the rec_df.
        leach_days_df.at[m_index,i] = days
        m_index += 1

In [5]:
for high in block_list:
    leach_days_df[high]=0
    lift_diff = 0
    block_leach_days(high,high,lift_diff)
    high_lift = blocks_records[high]['lift']
    for low in range(0,high):
        if lat_lng_list[high]==lat_lng_list[low]:
            low_lift = blocks_records[low]['lift']
            lift_diff = high_lift-low_lift
            block_leach_days(low,high,lift_diff)

In [6]:
print(leach_days_df[2000].tolist())
leach_days_records = leach_days_df.to_records()
print(leach_days_records['2000'].tolist())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 31, 31, 14, 31, 30, 31, 12, 0, 0, 28, 31, 30, 29, 0, 0, 10, 30, 31, 29, 0, 23, 30, 27, 24, 31, 31, 28, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 31, 31, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [42]:
# Prepare k factors
k = 0.0001
def k_calc():
    for b in block_list:
        leach_ks_df[b] = k
        leach_rec_df[b] = 0.0
    return leach_ks_df,leach_rec_df
leach_ks_df,leach_rec_df = k_calc()

# Prepare Rmax
rmax = blocks_df['rmax']

In [54]:
leach_ks_records = leach_ks_df.to_records(index=False)
leach_days_records = leach_days_df.to_records(index=False)
leach_rec_records = leach_rec_df.to_records(index=False)

In [44]:
for b in block_list:
    leach_ks_records[f'{b}'] + leach_days_records[f'{b}']

In [45]:

for b in block_list:
    rcum = 0
    b_rec = leach_rec_records[f'{b}']
    for m in range(0,len(monthlist)):
        R = recovery_calc(rmax[0],rcum,leach_ks_records[f'{b}'][m],leach_days_records[f'{b}'][m])
        b_rec[m] += R
        rcum += R

In [66]:
leach_days_records['2000']

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 26, 31, 31,  5,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0], dtype=int64)

In [58]:
leach_days_df[2000].tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 26,
 31,
 31,
 14,
 31,
 30,
 31,
 12,
 0,
 0,
 28,
 31,
 30,
 29,
 0,
 0,
 10,
 30,
 31,
 29,
 0,
 23,
 30,
 27,
 24,
 31,
 31,
 28,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]